In [1]:
import numpy as np
from scipy import spatial,stats
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
window_items =  [256, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 257, 17, 205, 206, 207, 210, 211, 84, 212, 213, 214, 215, 216, 217, 219, 220, 249, 233, 234, 235, 236, 239, 241, 243, 244, 245, 246, 247, 248, 121, 250, 251, 252, 253, 254, 255] 
target_items =  [100, 441, 453, 510, 592, 594, 694, 832, 1054] 

In [3]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation
/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation/../data/ratings.txt


In [34]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [35]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [36]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

In [37]:
test = pd.DataFrame()
for t_i in target_items:
    temp_df = train[train['itemId'] == t_i]
    train =  train.drop(train[train['itemId'] == t_i].index)
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)

In [38]:
test.shape

(77, 3)

### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [39]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 10)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3],dfs[4]], ignore_index=False)
del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [40]:
train.shape[0]

35449

In [41]:
test.shape[0] + train.shape[0] + del_data.shape[0]

35488

In [42]:
test.tail(5)

,userId,itemId,rating
27518,1157,592,0.5
3474,161,510,2.5
3333,161,453,2.5
1847,91,441,3.5
4273,188,100,3.0


In [43]:
train.tail(5)

,userId,itemId,rating
7941,323,592,1.5
6346,272,510,2.5
8003,323,453,0.5
19715,824,441,4.0
32493,1386,100,3.5


In [44]:
del_data.tail(5)

,userId,itemId,rating
28194,1187,510,2.0
28959,1212,453,0.5
24566,1034,441,0.5
21718,902,592,2.0
32486,1386,510,2.5


### Finding error for target items in actual data

In [45]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{592: [0, 0], 832: [0, 0], 594: [0, 0], 100: [0, 0], 453: [0, 0], 694: [0, 0], 441: [0, 0], 1054: [0, 0], 510: [0, 0]}


In [46]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
            if mov==441:
                print (sim)
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        print ( predicted,'  ',mov)
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

0    1054
0    832
0    694
0    594
0    592
0    592
0    510
0    510
0    453
0    441
3.86538461538    100
0    1054
0    832
0    694
0    594
0    592
0    592
0    510
0    453
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan    441
0    100


In [47]:
index

21

In [48]:
print (t2-t1)

2.4538092613220215


In [49]:
target_item_error_in_actual_data

{100: [3.8653846153846159, 2],
 441: [nan, 2],
 453: [3.4444444444444446, 2],
 510: [5.7272727272727275, 3],
 592: [7.0, 4],
 594: [3.8333333333333335, 2],
 694: [3.7142857142857144, 2],
 832: [3.7142857142857144, 2],
 1054: [3.625, 2]}

### MAE error

In [20]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

592     : 1.75
832     : 1.85714285714
594     : 1.91666666667
100     : 1.93269230769
453     : 1.72222222222
694     : 1.85714285714
441     : nan
1054     : 1.8125
510     : 1.90909090909


## Push Attack Strength Check

### Injecting Attack Profiles

In [21]:
push_rating = 4

In [22]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,push_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [23]:
train.shape[0]

41349

In [24]:
train.tail(5)

,userId,itemId,rating
41344,1699,592,4.0
41345,1699,594,4.0
41346,1699,694,4.0
41347,1699,832,4.0
41348,1699,1054,4.0


## Finding error for target items in attack profile injected data

In [25]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{592: [0, 0], 832: [0, 0], 594: [0, 0], 100: [0, 0], 453: [0, 0], 694: [0, 0], 441: [0, 0], 1054: [0, 0], 510: [0, 0]}


In [26]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [27]:
print (t2-t1)

3.012467384338379


In [28]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

592     : 1.75
832     : 1.85714285714
594     : 1.91666666667
100     : 1.93269230769
453     : 1.72222222222
694     : 1.85714285714
441     : nan
1054     : 1.8125
510     : 1.90909090909


In [ ]:
210     : 2.3058171134229086
235     : 2.1928669466604225
243     : 2.1907303256937363
244     : 2.349442613055623
248     : 2.4582805479544607
249     : 2.396745080071849
252     : 2.286436759922374
256     : 2.1500551206142404
257     : 2.4654589180088586
